In [2]:
import pandas as pd
import nltk
from nltk.metrics.agreement import AnnotationTask

In [3]:
df = pd.read_csv('~/Downloads/csv_les_republicain_summ.csv')

In [3]:
#df = pd.read_csv('~/git_repos/newsjam/classif/csv_les_republicain_summ.csv')

In [4]:
df

,date,relevance 1 (A),relevance 2 (J),relevance 3 (M),summary,text,title,topic,url
0,2021/10/14,1.0,0.0,0.0,Suivez avec nous l'évolution de la pandémie de...,22h55 : Le président letton atteint du Covid-1...,Santé. Covid-19 : ce qu'il faut retenir de la ...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...
1,2020/09/21,1.0,1.0,1.0,"Vaccination, taux d’incidence, nombre de patie...","En Bourgogne Franche-Comté, 180 patients sont ...",Infographies. Covid-19 : 1 décès supplémentair...,NaN,https://www.estrepublicain.fr/sante/2020/09/21...
2,2021/10/14,0.0,0.0,0.0,"Le laboratoire AstraZeneca, qui propose déjà u...",Des essais ont montré que le médicament AZD744...,Covid-19. Le régulateur européen évalue Evushe...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...
3,2021/10/14,0.0,0.0,0.0,L'OMS a dévoilé mercredi une nouvelle équipe d...,Les 26 membres ont été sélectionnés parmi 700 ...,Covid-19. Qui sont les nouveaux experts chargé...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...
4,2021/10/14,0.0,0.0,0.0,"Selon une étude américaine publiée mercredi, l...","Cette étude, financée par les Instituts nation...",Covid-19. Un rappel de Moderna ou Pfizer march...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...
...,...,...,...,...,...,...,...,...,...
1362,2021/11/08,NaN,NaN,NaN,"Ouverte depuis le 1er septembre dernier, la ca...","Plus de deux mois après son lancement, de nomb...",Infographies. Covid-19 : combien de personnes ...,NaN,https://www.estrepublicain.fr/sante/2021/11/08...
1363,2021/11/08,NaN,NaN,NaN,Elle frappe déjà les trois quarts du continent...,Le nombre de contaminations s'envole chaque se...,Santé. Covid-19 : cinq pistes pour limiter la ...,NaN,https://www.estrepublicain.fr/sante/2021/11/08...
1364,2021/11/08,NaN,NaN,NaN,"Dans la Meuse, le taux d’incidence au 5 novemb...","« Maintenir une distance physique avec autrui,...",Meuse. Covid-19 : le taux d’incidence proche d...,NaN,https://www.estrepublicain.fr/sante/2021/11/08...
1365,2021/11/03,NaN,NaN,NaN,Il s'agit des départements qui repassent au-de...,"""A partir de la semaine prochaine, de la rentr...",Epidémie. Covid-19 : voici les départements où...,NaN,https://www.estrepublicain.fr/sante/2021/11/03...


In [5]:
# So you can see how many columns you have filled in for the annotations (if you don't already know)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1367 entries, 0 to 1366
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             1367 non-null   object 
 1   relevance 1 (A)  691 non-null    float64
 2   relevance 2 (J)  693 non-null    float64
 3   relevance 3 (M)  693 non-null    float64
 4   summary          1367 non-null   object 
 5   text             1367 non-null   object 
 6   title            1367 non-null   object 
 7   topic            0 non-null      float64
 8   url              1367 non-null   object 
dtypes: float64(4), object(5)
memory usage: 96.2+ KB


In [6]:
def calc_iaa(df, num_anno, column1, column2):
    
    '''
    df = dataframe
    num_anno = number of annotations to add to the lists
    column1 = column with data from annotator 1
    column2 = column with data from annotator 2
    '''
    tags1 = [[] for x in range(num_anno)] # List of annotations from 1st annotator
    #for x in range(num_anno):
        #tags1.append([])
     
    # Creating list of all of the annotations with the correct format for nltk
    num = 0
    for i in tags1:
        i.append('A1')
        i.append(f'I{num + 1}')
        i.append(df[column1][num])
        num += 1
        
        
    tags2 = [[] for x in range(num_anno)] # List of annotations from 2nd annotator
    #for x in range(num_anno): 
        #tags2.append([])
        
    num = 0
    for i in tags2:
        i.append('A2')
        i.append(f'I{num + 1}')
        i.append(df[column2][num])
        num += 1
        
        
    # Combining the elements of both lists
    all_tags = tags1 + tags2
        
        
    # Instantiating AnnotationTask nltk function
    task = AnnotationTask(data=all_tags)
    
    # Calculating coefficients
    A0 = task.avg_Ao()
    S = task.S()
    pi = task.pi()
    kappa = task.kappa()

    coefficients = {}
    coefficients['A0'] = A0
    coefficients['S'] = S
    coefficients['pi'] = pi
    coefficients['kappa'] = kappa

    # Printing out the results
    for x in coefficients:
        print(f'{x}: {coefficients[x]}')

In [8]:
calc_iaa(df, 690, 'relevance 2 (J)', 'relevance 1 (A)')

A0: 0.8869565217391304
S: 0.8492753623188406
pi: 0.7415413872341038
kappa: 0.7415482135996926


In [9]:
'''
This block is just here to test everything and make sure the function is working properly at every step
'''

tags1 = [[] for x in range(690)] # List of annotations from 1st annotator
#for x in range(50):
    #tags1.append([])


# Creating list of all of the annotations with the correct format for nltk
num = 0
for i in tags1:
    i.append('A1')
    i.append(f'I{num + 1}')
    i.append(df['relevance 2 (J)'][num])
    num += 1
print(tags1)
print('\n')
tags2 = [[] for x in range(690)] # List of annotations from 2nd annotator
#for x in range(50): 
    #tags2.append([])


num = 0
for i in tags2:
    i.append('A2')
    i.append(f'I{num + 1}')
    i.append(df['relevance 3 (M)'][num])
    num += 1
print(tags2)
print('\n')

# Combining the elements of both lists
all_tags = tags1 + tags2

print(all_tags)

[['A1', 'I1', 0.0], ['A1', 'I2', 1.0], ['A1', 'I3', 0.0], ['A1', 'I4', 0.0], ['A1', 'I5', 0.0], ['A1', 'I6', 1.0], ['A1', 'I7', 1.0], ['A1', 'I8', 1.0], ['A1', 'I9', 0.0], ['A1', 'I10', 1.0], ['A1', 'I11', 0.0], ['A1', 'I12', 0.0], ['A1', 'I13', 0.0], ['A1', 'I14', 1.0], ['A1', 'I15', 1.0], ['A1', 'I16', 1.0], ['A1', 'I17', 0.0], ['A1', 'I18', 1.0], ['A1', 'I19', 1.0], ['A1', 'I20', 0.0], ['A1', 'I21', 1.0], ['A1', 'I22', 1.0], ['A1', 'I23', 0.0], ['A1', 'I24', 1.0], ['A1', 'I25', 1.0], ['A1', 'I26', 0.0], ['A1', 'I27', 1.0], ['A1', 'I28', 1.0], ['A1', 'I29', 0.0], ['A1', 'I30', 1.0], ['A1', 'I31', 0.0], ['A1', 'I32', 1.0], ['A1', 'I33', 0.0], ['A1', 'I34', 1.0], ['A1', 'I35', 0.0], ['A1', 'I36', 1.0], ['A1', 'I37', 1.0], ['A1', 'I38', 0.0], ['A1', 'I39', 1.0], ['A1', 'I40', 1.0], ['A1', 'I41', 1.0], ['A1', 'I42', 0.0], ['A1', 'I43', 0.0], ['A1', 'I44', 0.0], ['A1', 'I45', 0.0], ['A1', 'I46', 0.0], ['A1', 'I47', 0.0], ['A1', 'I48', 1.0], ['A1', 'I49', 1.0], ['A1', 'I50', 1.0], ['A1', '